In [13]:
# tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-10-06 16:18:15--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: 'input.txt.1'

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.1s    

2023-10-06 16:18:15 (7.24 MB/s) - 'input.txt.1' saved [1115394/1115394]



In [2]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print('len',len(text))

len 1115394


In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

### Tokenizer, Encoder, Decoder

In [4]:
stoi = {char:i for i,char in enumerate(chars)} #string to int
itos = {i:char for i,char in enumerate(chars)} #int to string
encode = lambda input_str: [stoi[char] for char in input_str]
decode = lambda input_enc: ''.join([itos[enc] for enc in input_enc])

In [5]:
encode('hello world')

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]

In [6]:
decode(encode('hello world'))

'hello world'

### Tokenize and put into torch tensor

In [7]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
data[:20]

torch.Size([1115394]) torch.int64


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56])

### Spliting

In [8]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [9]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when context: {context}, target: {target}, t:{t}")

when context: tensor([18]), target: 47, t:0
when context: tensor([18, 47]), target: 56, t:1
when context: tensor([18, 47, 56]), target: 57, t:2
when context: tensor([18, 47, 56, 57]), target: 58, t:3
when context: tensor([18, 47, 56, 57, 58]), target: 1, t:4
when context: tensor([18, 47, 56, 57, 58,  1]), target: 15, t:5
when context: tensor([18, 47, 56, 57, 58,  1, 15]), target: 47, t:6
when context: tensor([18, 47, 56, 57, 58,  1, 15, 47]), target: 58, t:7


In [11]:
torch.manual_seed(1337)
batch_size = 4 # independent sequences
block_size = 8

def get_batch(split):
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])

    print('ix: ', ix)
    return x,y

In [12]:
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print()
print('targets:')
print(yb.shape)
print(yb)

ix:  tensor([ 76049, 234249, 934904, 560986])
inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])

targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [13]:
for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53, 56, 1, 58, 46] the target: 39
when input is [44, 53, 56, 1, 58, 46, 39] the target: 58
when input is [44, 53, 56, 1, 58, 46, 39, 58] the target: 1
when input is [52] the target: 58
when input is [52, 58] the target: 1
when input is [52, 58, 1] the target: 58
when input is [52, 58, 1, 58] the target: 46
when input is [52, 58, 1, 58, 46] the target: 39
when input is [52, 58, 1, 58, 46, 39] the t